<a href="https://colab.research.google.com/github/hiddenguy1/Awesome-pytorch-list/blob/master/test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Initial Setup

In [ ]:
!pip install torch numpy transformers sklearn 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 47.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 106.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 24.8 MB/s eta 0:00:00
  Created wheel for sklearn: filename=sklearn-0.0.post4-py3-none-any.whl size=2973 sha256=10d6b13ea69234500f5aa4d4b2873bad42eaa7059c2c5563541ec1156e80bb04
  Stored in directory: /root/.cache/pip/wheels/46/b8/69/4f7789b7d2fc0718a44cbc4ae5d40970a40c22c472ae50f0a1
Successfully built sklearn


In [ ]:
! pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 16.9 MB/s eta 0:00:00


In [ ]:
import torch
torch.cuda.is_available()
device = torch.device("cuda")

In [ ]:
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

# set a seed value
torch.manual_seed(555)

from sklearn.utils import shuffle
from sklearn.metrics import roc_auc_score, accuracy_score

import transformers
from transformers import BertTokenizer, BertForSequenceClassification 
from transformers import XLMRobertaTokenizer, XLMRobertaForSequenceClassification, XLMRobertaModel
from transformers import AdamW

In [ ]:
from transformers import pipeline
unmasker = pipeline('fill-mask', model='xlm-roberta-base')
unmasker("Hello I'm a <mask> model.")
from transformers import AutoTokenizer, AutoModelForMaskedLM

rTokenizer = AutoTokenizer.from_pretrained('xlm-roberta-base')
model = AutoModelForMaskedLM.from_pretrained("xlm-roberta-base")
# rTokenizer = XLMRobertaTokenizer.from_pretrained('xlm-roberta-base')

In [ ]:
print(rTokenizer.vocab_size)

250002


# Data

## Import


In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
import gc

In [ ]:
urduRaw = pd.read_csv("urdu.tsv", sep='\t')
spanishRaw = pd.read_csv("spanish.csv")
marathiRaw = pd.read_csv("marathi.csv")
chineseRaw = pd.read_csv("chinese.csv")
englishRaw = pd.read_csv("english.tsv", sep='\t')

## Cleaning
### Spanish

In [ ]:
spanish = spanishRaw[["text", "airline_sentiment"]].copy(deep=True).sample(frac=1).reset_index(drop=True)
spanish.rename(columns = {'airline_sentiment':'label'}, inplace = True)
spanish = spanish[spanish['label']!="neutral"]
spanish.loc[(spanish.label == "negative"), "label"] = 0
spanish.loc[(spanish.label == 'positive'), "label"] = 1
spanish['language']='spanish'

In [ ]:
spanish['label'].value_counts()

0    3769
1    1489
Name: label, dtype: int64

In [ ]:
val_sizeS = int(spanish.shape[0] * 0.15)
test_sizeS = int(spanish.shape[0] * 0.15)
test_spanish = spanish[:test_sizeS].reset_index(drop=True)
val_spanish = spanish[test_sizeS:test_sizeS+val_sizeS].reset_index(drop=True)
train_spanish = spanish[test_sizeS+val_sizeS:].reset_index(drop=True)

### Urdu

In [ ]:
urdu=urduRaw.copy(deep=True).sample(frac=1).reset_index(drop=True)
urdu.rename(columns = {'Tweet':'text','Class':'label'}, inplace = True)
urdu = urdu[urdu['label'].notna()]
urdu = urdu[urdu['label']!='O']
urdu.loc[(urdu.label == "N"), "label"] = 0
urdu.loc[(urdu.label == 'P'), "label"] = 1
urdu['language']='urdu'

In [ ]:
urdu['label'].value_counts()

0    499
1    480
Name: label, dtype: int64

In [ ]:
val_sizeU = int(urdu.shape[0] * 0.15)
test_sizeU = int(urdu.shape[0] * 0.15)
test_urdu = urdu[:test_sizeU].reset_index(drop=True)
val_urdu = urdu[test_sizeU:test_sizeU+val_sizeU].reset_index(drop=True)
train_urdu = urdu[test_sizeU+val_sizeU:].reset_index(drop=True)

### Marathi

In [ ]:
marathi=marathiRaw.copy(deep=True).sample(frac=1).reset_index(drop=True)
marathi.rename(columns = {'tweet':'text'}, inplace = True)
marathi = marathi[marathi['label']!=0]
marathi.loc[(marathi.label == -1), "label"] = 0
marathi['language']='marathi'

In [ ]:
marathi['label'].value_counts()

0    4038
1    4038
Name: label, dtype: int64

In [ ]:
val_sizeM = int(marathi.shape[0] * 0.15)
test_sizeM = int(marathi.shape[0] * 0.15)
test_marathi = marathi[:test_sizeM].reset_index(drop=True)
val_marathi = marathi[test_sizeM:test_sizeM+val_sizeM].reset_index(drop=True)
train_marathi = marathi[test_sizeM+val_sizeM:].reset_index(drop=True)

### Chinese

In [ ]:
chinese = chineseRaw[["review", "label"]].copy(deep=True).sample(frac=1).reset_index(drop=True)
chinese.rename(columns = {'review':'text'}, inplace = True)

In [ ]:
chinese['language']='chinese'

In [ ]:
chinese = chinese.sample(9000).reset_index(drop=True).copy()
chinese['label'].value_counts()

1    4501
0    4499
Name: label, dtype: int64

In [ ]:
val_sizeC = int(chinese.shape[0] * 0.15)
test_sizeC = int(chinese.shape[0] * 0.15)
test_chinese = chinese[:test_sizeC].reset_index(drop=True)
val_chinese = chinese[test_sizeC:test_sizeC+val_sizeC].reset_index(drop=True)
train_chinese = chinese[test_sizeC+val_sizeC:].reset_index(drop=True)

### English

In [ ]:
english = englishRaw[["sentence", "label"]].copy(deep=True).sample(frac=1).reset_index(drop=True)
english.rename(columns = {'sentence':'text'}, inplace = True)
english['language']='english'

In [ ]:
english = english.sample(9000).reset_index(drop=True).copy()
english['label'].value_counts()

1    4552
0    4448
Name: label, dtype: int64

In [ ]:
val_sizeE = int(english.shape[0] * 0.15)
test_sizeE = int(english.shape[0] * 0.15)
test_english = english[:test_sizeE].reset_index(drop=True)
val_english = english[test_sizeE:test_sizeE+val_sizeE].reset_index(drop=True)
train_english = english[test_sizeE+val_sizeE:].reset_index(drop=True)

In [ ]:
pd.concat([chinese[0:1],english[0:1],spanish[0:1],marathi[0:1],urdu[0:1]])

,text,label,language
0,又是一例虐童！越来越没底线了。 //@姚晨:太可憎了！这些教师脑子到底都在想什么！！！[怒]...,0,chinese
0,"I don't normally write reviews, but this ""film...",0,english
0,@Iberia @pepillogrillo67 @JosPastr +Esa señora...,0,spanish
0,नरेंद्र मोदींना राहुल गांधी यांनी सरेंडर मोद...,0,marathi
0,بجلی کی قیمتوں میں اضافہ واپس,1,urdu


## Combine and Summary

In [ ]:
test = pd.concat([test_english,test_chinese,test_urdu, test_spanish, test_marathi]).sample(frac=1).reset_index(drop=True)
val = pd.concat([val_english,val_chinese,val_urdu, val_spanish, val_marathi]).sample(frac=1).reset_index(drop=True)
train = pd.concat([train_english,train_chinese,train_urdu, train_spanish, train_marathi]).sample(frac=1).reset_index(drop=True)

In [ ]:
testSummary = pd.DataFrame(columns = test['language'].unique())
temp=[0,0,0,0,0]
testSummary.loc['positive']=temp
testSummary.loc['negative']=temp
testSummary.loc['total']=temp
for lang in test['language'].unique():
  pos=test.loc[(test.language == lang)]['label'].value_counts()[1]
  neg=test.loc[(test.language == lang)]['label'].value_counts()[0]
  testSummary.loc['positive'][lang]=pos
  testSummary.loc['negative'][lang]=neg
  testSummary.loc['total'][lang]=pos+neg
testSummary['total'] = testSummary.sum(axis=1)
testSummary

,spanish,chinese,english,marathi,urdu,total
positive,242,692,683,607,67,2291
negative,546,658,667,604,79,2554
total,788,1350,1350,1211,146,4845


In [ ]:
valSummary = pd.DataFrame(columns = val['language'].unique())
temp=[0,0,0,0,0]
valSummary.loc['positive']=temp
valSummary.loc['negative']=temp
valSummary.loc['total']=temp
for lang in val['language'].unique():
  pos=val.loc[(val.language == lang)]['label'].value_counts()[1]
  neg=val.loc[(val.language == lang)]['label'].value_counts()[0]
  valSummary.loc['positive'][lang]=pos
  valSummary.loc['negative'][lang]=neg
  valSummary.loc['total'][lang]=pos+neg
valSummary['total'] = valSummary.sum(axis=1)
valSummary

,chinese,marathi,spanish,english,urdu,total
positive,684,600,212,679,74,2249
negative,666,611,576,671,72,2596
total,1350,1211,788,1350,146,4845


In [ ]:
trainSummary = pd.DataFrame(columns = train['language'].unique())
temp=[0,0,0,0,0]
trainSummary.loc['positive']=temp
trainSummary.loc['negative']=temp
trainSummary.loc['total']=temp
for lang in train['language'].unique():
  pos=train.loc[(train.language == lang)]['label'].value_counts()[1]
  neg=train.loc[(train.language == lang)]['label'].value_counts()[0]
  trainSummary.loc['positive'][lang]=pos
  trainSummary.loc['negative'][lang]=neg
  trainSummary.loc['total'][lang]=pos+neg
trainSummary['total'] = trainSummary.sum(axis=1)
trainSummary

,marathi,english,urdu,chinese,spanish,total
positive,2831,3190,339,3125,1035,10520
negative,2823,3110,348,3175,2647,12103
total,5654,6300,687,6300,3682,22623


In [ ]:
test_texts, test_labels = list(test.text), list(test.label)
val_texts, val_labels = list(val.text), list(val.label)
train_texts, train_labels = list(train.text), list(train.label)

# Model Setup

## Define parameters

In [ ]:
import os
# Parameters for fine-tuning
L_RATE = 3e-5
MAX_LEN = 20
NUM_EPOCHS = 3
BATCH_SIZE = 64
NUM_CORES = os.cpu_count()
NUM_CORES

12

In [ ]:
model = XLMRobertaForSequenceClassification.from_pretrained(
    'xlm-roberta-base', 
    num_labels = 2,
)
model.to(device)

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'roberta.pooler.dense.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.out_p

XLMRobertaForSequenceClassification(
  (roberta): XLMRobertaModel(
    (embeddings): XLMRobertaEmbeddings(
      (word_embeddings): Embedding(250002, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): XLMRobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x XLMRobertaLayer(
          (attention): XLMRobertaAttention(
            (self): XLMRobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): XLMRobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768,

## Tokenization

In [ ]:
class CompDataset(Dataset):
  
    def __init__(self, df):
        self.df_data = df

    def __getitem__(self, index):
        text = self.df_data.loc[index, 'text']
        # tokenization
        encoded_dict = rTokenizer.encode_plus(
                    text,
                    max_length = MAX_LEN,
                    pad_to_max_length = True,
                    return_attention_mask = True,
                    return_tensors = 'pt',
               )
        padded_token_list = encoded_dict['input_ids'][0]
        att_mask = encoded_dict['attention_mask'][0]
        target = torch.tensor(self.df_data.loc[index, 'label'])
        sample = (padded_token_list, att_mask, target)
        return sample

    def __len__(self):
        return len(self.df_data)

In [ ]:
test_data = CompDataset(test)
train_data = CompDataset(train)
val_data = CompDataset(val)

In [ ]:
print(len(test_data))
print(len(train_data))
print(len(val_data))

4845
22623
4845


# Baseline Results

In [ ]:
def createBaseline(data, name):
    test_data = CompDataset(data)
    dataloader = torch.utils.data.DataLoader(test_data,
                      batch_size=BATCH_SIZE,
                      shuffle=True,
                      num_workers=NUM_CORES)
    model.eval()
    torch.set_grad_enabled(False)
    targets_list=[]
    for j, batch in enumerate(dataloader):       
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)      
        outputs = model(b_input_ids, 
                attention_mask=b_input_mask, 
                labels=b_labels)        
        loss = outputs[0]
        preds = outputs[1]
        val_preds = preds.detach().cpu().numpy()
        targets_np = b_labels.to('cpu').numpy()
        targets_list.extend(targets_np)
        if j == 0: 
            stacked_val_preds = val_preds
        else:
            stacked_val_preds = np.vstack((stacked_val_preds, val_preds))  
    y_true = targets_list
    y_pred = np.argmax(stacked_val_preds, axis=1)
    acc = accuracy_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    print(name,' test acc: ', acc)
    print(name, ': test F1: ', f1)

In [ ]:
createBaseline(test_chinese, 'Chinese')
createBaseline(test_english, 'English')
createBaseline(test_urdu, 'Urdu')
createBaseline(test_marathi, 'Marathi')
createBaseline(test_spanish, 'Spanish')

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2354: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest s

Chinese  test acc:  0.5125925925925926
Chinese : test F1:  0.6777668952007836


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-s

English  test acc:  0.5059259259259259
English : test F1:  0.6719134284308903


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2354: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest s

Urdu  test acc:  0.4589041095890411
Urdu : test F1:  0.6291079812206573


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2354: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest s

Marathi  test acc:  0.5012386457473162
Marathi : test F1:  0.6677667766776677


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2354: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest s

Spanish  test acc:  0.30710659898477155
Spanish : test F1:  0.46990291262135914


# Fine-tune

In [ ]:
import random
import gc


In [ ]:
print(model.parameters())

<generator object Module.parameters at 0x7fa76511a6c0>


In [ ]:
optimizer = AdamW(model.parameters(),
              lr = L_RATE, 
              eps = 1e-8 
            )

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
train_data = CompDataset(train_marathi)
val_data = CompDataset(val)

In [ ]:
train_dataloader = torch.utils.data.DataLoader(train_data,
                    batch_size=BATCH_SIZE,
                    shuffle=True,
                    num_workers=NUM_CORES)
val_dataloader = torch.utils.data.DataLoader(val_data,
                    batch_size=BATCH_SIZE,
                    shuffle=True,
                    num_workers=NUM_CORES)

In [ ]:
gc.collect()

87

In [ ]:
seed_val = 101

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)
loss_values = []

for epoch in range(0, NUM_EPOCHS):
    
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch + 1, NUM_EPOCHS))
    
    stacked_val_labels = []
    targets_list = []

    print('Training...')
    model.train()
    torch.set_grad_enabled(True)
    total_train_loss = 0
    gc.collect()

    for i, batch in enumerate(train_dataloader):
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        model.zero_grad()        
        outputs = model(b_input_ids, 
                    attention_mask=b_input_mask,
                    labels=b_labels)
        loss = outputs[0]
        total_train_loss = total_train_loss + loss.item()
        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()   
        gc.collect()

    print('Train loss:' ,total_train_loss)

    gc.collect()

    print('\nValidation...')
    model.eval()
    torch.set_grad_enabled(False)
    total_val_loss = 0
    for j, batch in enumerate(val_dataloader):       
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)      
        outputs = model(b_input_ids, 
                attention_mask=b_input_mask, 
                labels=b_labels)        
        loss = outputs[0]
        total_val_loss = total_val_loss + loss.item()
        preds = outputs[1]
        val_preds = preds.detach().cpu().numpy()
        targets_np = b_labels.to('cpu').numpy()
        targets_list.extend(targets_np)
        if j == 0:  # first batch
            stacked_val_preds = val_preds
        else:
            stacked_val_preds = np.vstack((stacked_val_preds, val_preds))  
        gc.collect()
    y_true = targets_list
    y_pred = np.argmax(stacked_val_preds, axis=1)
    val_acc = accuracy_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    print('Val loss:' ,total_val_loss)
    print('Val acc: ', val_acc)
    print('Val f1:' ,f1)
    gc.collect()

    torch.save(model.state_dict(), 'model.pt')

    gc.collect()

        


======== Epoch 1 / 3 ========
Training...


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-s

Train loss: 51.495517164468765

Validation...


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2354: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `

Val loss: 47.728451400995255
Val acc:  0.7009287925696595
Val f1: 0.6473594548551959

======== Epoch 2 / 3 ========
Training...


...

Val loss: 44.62039041519165
Val acc:  0.709391124871001
Val f1: 0.6327595200834639

======== Epoch 3 / 3 ========
Training...


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-s

Train loss: 33.23702718317509

Validation...


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-s

Val loss: 51.81259232759476
Val acc:  0.7056759545923632
Val f1: 0.6660421545667448


In [ ]:
gc.collect()

0

# Result and Anlaysis

In [ ]:

def singleLangTest(data, name):
    test_data = CompDataset(data)
    dataloader = torch.utils.data.DataLoader(test_data,
                      batch_size=BATCH_SIZE,
                      shuffle=True,
                      num_workers=NUM_CORES)
    model.eval()
    torch.set_grad_enabled(False)
    targets_list=[]
    for j, batch in enumerate(dataloader):       
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)      
        outputs = model(b_input_ids, 
                attention_mask=b_input_mask, 
                labels=b_labels)        
        loss = outputs[0]
        preds = outputs[1]
        val_preds = preds.detach().cpu().numpy()
        targets_np = b_labels.to('cpu').numpy()
        targets_list.extend(targets_np)
        if j == 0: 
            stacked_val_preds = val_preds
        else:
            stacked_val_preds = np.vstack((stacked_val_preds, val_preds))  
    y_true = targets_list
    y_pred = np.argmax(stacked_val_preds, axis=1)
    acc = accuracy_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    # print(name)
    print(name,'test acc: ', acc)
    print(name,'test F1: ', f1)
    print(name,'precision ', precision)
    print(name,'recall ', recall)


In [ ]:
singleLangTest(test_chinese,'chinese')

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2354: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest s

chinese test acc:  0.654074074074074
chinese test F1:  0.5899912203687445
chinese precision  0.7516778523489933
chinese recall  0.48554913294797686


In [ ]:
singleLangTest(test_english,'english')

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-s

english test acc:  0.6622222222222223
english test F1:  0.6409448818897637
english precision  0.6933560477001703
english recall  0.595900439238653


In [ ]:
singleLangTest(test_spanish,'spanish')

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2354: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest s

spanish test acc:  0.7474619289340102
spanish test F1:  0.6012024048096193
spanish precision  0.5836575875486382
spanish recall  0.6198347107438017


In [ ]:
singleLangTest(test_urdu,'urdu')

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-s

urdu test acc:  0.684931506849315
urdu test F1:  0.6290322580645161
urdu precision  0.6842105263157895
urdu recall  0.582089552238806


In [ ]:
singleLangTest(test_marathi,'marathi')

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2354: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest s

marathi test acc:  0.8117258464079273
marathi test F1:  0.8243451463790447
marathi precision  0.7742402315484804
marathi recall  0.8813838550247117
